In [ ]:
%load_ext autoreload
%autoreload 2
from arpes.all import *
import os.path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import numpy as np
%matplotlib inline

In [ ]:
# Define directory
path = os.path.abspath("G:\My Drive\Box_1_square_renamed")

# Different parameters used to set up scans at SLS
scan_type = '*'
direction = 'X'
phi = '*'
energy = '25eV'
polarization = '*'
temp = '*'
e_pass = '*'+'eVpass'
comment = '*'
run = '*'.zfill(4)

# Put together string for finding file
parameters = [scan_type, direction, phi, energy, polarization, temp, e_pass, comment, run]
if comment == '':
    parameters.remove(comment)
file_type = 'h5'
file_name = '_'.join(parameters)
file = '.'.join([file_name , file_type])

au_full_file = os.path.join(path, file)
file

In [ ]:
import glob
# Load all files with above specified parameters, * is a placeholder and imports all files in its place
au_files = glob.glob(au_full_file)
print(len(au_files))
au_files

In [4]:
au_raw_data_list = {}
for file in au_files:
    #data_list.append(load_data(file, location="ULTRA"))
    data = load_data(file, location="ULTRA")
    # k_data = convert_to_kspace(data.spectrum)
    file_name = file.replace(path, '')[1:-3]
    au_raw_data_list.update({file_name : data})

poly Au used to determine Fermi level. Determine Fermiedge and apply to data below

In [ ]:
au_lt = au_raw_data_list['EDC_X_P95_25eV_UNKN_LT_5eVpass_GoldRef11K_0007']
au_lt.S.plot()

In [ ]:
from arpes.analysis.general import rebin

binned = rebin(au_lt, phi=10).spectrum
binned.plot()

In [ ]:
from arpes.all import AffineBroadenedFD, broadcast_model
fermi_level = binned.sel(phi=slice(-0.2,0.2), eV=slice(-0.2,0.1))
results = broadcast_model(AffineBroadenedFD, fermi_level, "phi")

fig, ax = plt.subplots()
fermi_level.plot(ax=ax)
ax.scatter(*results.F.p("fd_center").G.to_arrays(), color="red")
ax.axhline(y=0, color='k', linestyle='--', linewidth=1)

In [ ]:
from arpes.fits.fit_models import LinearModel
lin_mod = LinearModel().guess_fit(results.F.p("fd_center"))
lin_mod.plot()
plt.show()

In [ ]:
edc = au_lt.spectrum
edge = lin_mod.eval(x=edc.phi)

corrected_edc = edc.G.shift_by(edge, "eV")

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
corrected_edc.plot(ax=axes[0], vmax=10)
edc.plot(ax=axes[1], vmax=20)
# plot vertical lines for y = 0
for ax in axes:
    ax.set_title("")
    ax.axhline(y=0, color='k', linestyle='--', linewidth=1)

In [ ]:
# Define directory
path = os.path.abspath("G:\My Drive\Box_1_square_renamed")

# Different parameters used to set up scans at SLS
scan_type = 'FS'
direction = 'N'
phi = 'P50'
energy = '*'
polarization = 'CIRCP'
temp = 'LT'
e_pass = '*'+'eVpass'
comment = ''
run = '*'.zfill(4)

# Put together string for finding file
parameters = [scan_type, direction, phi, energy, polarization, temp, e_pass, comment, run]
if comment == '':
    parameters.remove(comment)
file_type = 'h5'
file_name = '_'.join(parameters)
file = '.'.join([file_name , file_type])

bi2223_full_file = os.path.join(path, file)
file

In [ ]:
import glob
# Load all files with above specified parameters, * is a placeholder and imports all files in its place
bi2223_files = glob.glob(bi2223_full_file)
print(len(bi2223_files))
bi2223_files

In [12]:
bi2223_data_list = {}
for file in bi2223_files:
    #data_list.append(load_data(file, location="ULTRA"))
    raw_data = load_data(file, location="ULTRA")
    edge = lin_mod.eval(x=raw_data.spectrum.phi)
    corrected_data = raw_data.spectrum.G.shift_by(edge, "eV")
    corrected_data.S.apply_offsets({"beta": -0.0077, "phi": -0.004})
    k_data = convert_to_kspace(corrected_data)
    file_name = file.replace(path, '')[1:-3]
    bi2223_data_list.update({file_name : k_data})

Initalize datasets

In [14]:
lt_n_p5 = bi2223_data_list['FS_N_P50_26p5eV_CIRCP_LT_10eVpass_0000']


Overview of the LT P5 Dataset

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
lt_n_p5.sel(eV=slice(-0.02,0.02)).sum("eV").plot(ax=ax, cmap='viridis')

# plt.ylim(-0.56, 0)
# plt.xlim(-0, 0.56)
plt.title('')
plt.ylabel('$k_y$ ($\AA^{-1}$)', fontsize=18)
plt.xlabel('$k_x$ ($\AA^{-1}$)', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=16)
# plt.yticks(np.arange(-0.4, 0.5, 0.2))
# plt.xticks(np.arange(-0.4, 0.5, 0.2))

kxs = [-0.55, -0.5, -0.45, -0.4, -0.35, -0.30, -0.25, -0.22, -0.2]
kys = [0.16, 0.18, 0.21, 0.24, 0.27, 0.32, 0.38, 0.44, 0.49]
for kx, ky in zip(kxs, kys):
    plt.scatter(kx, ky, color='red', s=50, marker='x')

ax.collections[0].colorbar.remove()
cbar = plt.colorbar(ax.collections[0], ax=ax)
cbar.set_label('Intensity', fontsize=16, labelpad=5)
cbar.set_ticks([])
# set x and y axis to be equal
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [32]:
def normalize(data):
    return (data - data.sel(eV=slice(0.05,0.07)).mean("eV")) / max(data.values)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))

edc_an = lt_an_p5.sel(eV=slice(-0.24,None), kx=slice(-0.58,-0.52), ky=slice(0.13,0.19)).sum("kx").sum("ky")
edc_n = lt_an_p5.sel(eV=slice(-0.24,None), kx=slice(-0.33,-0.28), ky=slice(0.28,0.31)).sum("kx").sum("ky")
au_ref =corrected_edc.sel(eV=slice(-0.24,None), phi=slice(-0.05,0.05)).sum("phi")

normalize(au_ref).plot(ax=ax, label='Au')
normalize(edc_n).plot(ax=ax, label='N')
normalize(edc_an).plot(ax=ax, label='AN')

plt.legend()

In [ ]:
fig, (ax, ax2) = plt.subplots(1, 2, figsize=(10, 10))

cmap = plt.get_cmap('copper')
colors = [cmap(i) for i in np.linspace(0, 1, len(kxs)+3)]

i=0
for kx, ky in zip(kxs, kys):
    edc = lt_an_p5.sel(eV=slice(-0.1,0.1), kx=slice(kx-0.01,kx+0.01), ky=slice(ky-0.01,ky+0.01)).sum("kx").sum("ky")
    (normalize(edc)+i*0.8).sel(eV=slice(-0.08,0)).plot(ax=ax, label=f'({kx}, {ky})', color=colors[i+1])
    ax.plot(-1*edc.sel(eV=slice(-0.08,0)).eV.values, (normalize(edc)+i*0.8).sel(eV=slice(-0.08,0)).values, color=colors[i+1])
    i+=1

i=0
for kx, ky in zip(kxs, kys):
    edc = ht_an_p5.sel(eV=slice(-0.1,0.1), kx=slice(kx-0.01,kx+0.01), ky=slice(ky-0.01,ky+0.01)).sum("kx").sum("ky")
    (normalize(edc)+i*0.8).sel(eV=slice(-0.08,0)).plot(ax=ax2, label=f'({kx}, {ky})', color=colors[i+1])
    ax2.plot(-1*edc.sel(eV=slice(-0.08,0)).eV.values, (normalize(edc)+i*0.8).sel(eV=slice(-0.08,0)).values, color=colors[i+1])
    i+=1

for axis in [ax, ax2]:
    axis.set_title('')
    axis.set_ylabel('Intensity')
    axis.set_xlabel('Energy (eV)')


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
lt_an_p5.sel(eV=slice(-0.02,0.02)).sum("eV").plot(ax=ax, cmap='viridis', vmin=20, vmax=100)

# plt.ylim(-0.4, 0.4)
# plt.xlim(0.0, 0.55)
plt.title('')
plt.ylabel('$k_y$ ($\AA^{-1}$)', fontsize=18)
plt.xlabel('$k_x$ ($\AA^{-1}$)', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=16)


ax.collections[0].colorbar.remove()
cbar = plt.colorbar(ax.collections[0], ax=ax)
cbar.set_label('Intensity', fontsize=16, labelpad=5)
cbar.set_ticks([])
# set x and y axis to be equal
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))

edm = lt_an_p5.sel(kx=slice(-0.55,0), eV=slice(-0.08,0.02), ky=slice(-0.48,-0.42)).sum("ky")
edm.plot(ax=ax, vmin=3, vmax=10, cmap='PuBu_r')

In [ ]:

fig, ax = plt.subplots(3, 5, figsize=(20, 15))

kxs = np.arange(-0.35, 0.36, 0.05)

for i, kx in enumerate(kxs):
    edm = lt_n_p5.sel(ky=slice(0.2,0.5), eV=slice(-0.08,0.02), kx=slice(kx-0.02,kx+0.02)).sum("kx")
    edm.plot(ax=ax.flat[i])
    ax.flat[i].set_title(f'ky={np.round(kx,2)}')


plt.tight_layout()

In [ ]:

fig, ax = plt.subplots(3, 5, figsize=(20, 15))

kxs = np.arange(-0.35, 0.36, 0.05)

for i, kx in enumerate(kxs):
    edm = lt_n_p5.sel(ky=slice(-0.6,-0.4), eV=slice(-0.08,0.02), kx=slice(kx-0.02,kx+0.02)).sum("kx")
    edm.plot(ax=ax.flat[i])
    ax.flat[i].set_title(f'ky={np.round(kx,2)}')


plt.tight_layout()